In [2]:
# Instalar ultralytics (asegúrate de haber ejecutado esta celda antes)
!pip install ultralytics

# Importar YOLO y comprobar la instalación
from ultralytics import YOLO

# Verificar la instalación (sin `checks()`)
!yolo help



    Arguments received: ['yolo', 'help']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of {'segment', 'pose', 'obb', 'classify', 'detect'}
                MODE (required) is one of {'track', 'val', 'export', 'train', 'benchmark', 'predict'}
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolo11n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at image size 320:
        yolo predict model=yolo11n-seg.pt source='https://youtu.be/LNwODJXcvt4' imgsz=320

    3. Val a pretrained detection model at batch-size 1 and image size 640:
        yolo val model=yolo11n.pt data=coco8.yaml batch=1 i

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
import os
import shutil

# Ruta a los datos en Google Drive
base_path = "/content/drive/MyDrive/YOLOv8/datasets"

# Combinar carpetas de imágenes y etiquetas para entrenamiento
for category in ['train', 'val']:
    for subfolder in ['correct', 'incorrect']:
        images_src = os.path.join(base_path, category, 'images', subfolder)
        labels_src = os.path.join(base_path, category, 'labels', subfolder)
        images_dst = os.path.join(base_path, category, 'images')
        labels_dst = os.path.join(base_path, category, 'labels')

        if os.path.exists(images_src):
            for file in os.listdir(images_src):
                shutil.move(os.path.join(images_src, file), images_dst)
        if os.path.exists(labels_src):
            for file in os.listdir(labels_src):
                shutil.move(os.path.join(labels_src, file), labels_dst)


In [7]:
import os

# Ruta base donde están los archivos de etiquetas
train_labels_path = "/content/drive/MyDrive/YOLOv8/datasets/train/labels"
val_labels_path = "/content/drive/MyDrive/YOLOv8/datasets/val/labels"

# Función para obtener todas las clases utilizadas
def get_classes_from_labels(labels_path):
    classes = set()
    for file in os.listdir(labels_path):
        if file.endswith(".txt"):  # Asegúrate de solo leer archivos .txt
            with open(os.path.join(labels_path, file), 'r') as f:
                for line in f:
                    class_id = int(line.split()[0])  # La primera columna es la clase
                    classes.add(class_id)
    return classes

# Obtener clases únicas en el conjunto de entrenamiento y validación
train_classes = get_classes_from_labels(train_labels_path)
val_classes = get_classes_from_labels(val_labels_path)

# Combinar todas las clases detectadas
all_classes = sorted(train_classes.union(val_classes))

print("Clases detectadas en el conjunto de entrenamiento:", sorted(train_classes))
print("Clases detectadas en el conjunto de validación:", sorted(val_classes))
print("Total de clases detectadas:", len(all_classes))
print("Clases detectadas (ordenadas):", all_classes)

# Generar un archivo dataset.yaml
dataset_yaml = f"""
train: /content/drive/MyDrive/YOLOv8/datasets/train/images
val: /content/drive/MyDrive/YOLOv8/datasets/val/images

nc: {len(all_classes)}
names: {["Clase_" + str(cls) for cls in all_classes]}  # Aquí puedes reemplazar con nombres reales si los conoces
"""

# Guardar el archivo dataset.yaml
yaml_path = "/content/drive/MyDrive/YOLOv8/dataset.yaml"
with open(yaml_path, 'w') as f:
    f.write(dataset_yaml)

print(f"\nArchivo dataset.yaml generado en: {yaml_path}")
print("\nContenido sugerido para dataset.yaml:")
print(dataset_yaml)


Clases detectadas en el conjunto de entrenamiento: [0, 1, 2, 3, 4]
Clases detectadas en el conjunto de validación: [0, 1, 2, 3, 4]
Total de clases detectadas: 5
Clases detectadas (ordenadas): [0, 1, 2, 3, 4]

Archivo dataset.yaml generado en: /content/drive/MyDrive/YOLOv8/dataset.yaml

Contenido sugerido para dataset.yaml:

train: /content/drive/MyDrive/YOLOv8/datasets/train/images
val: /content/drive/MyDrive/YOLOv8/datasets/val/images

nc: 5
names: ['Clase_0', 'Clase_1', 'Clase_2', 'Clase_3', 'Clase_4']  # Aquí puedes reemplazar con nombres reales si los conoces



In [ ]:
# Entrenar YOLOv8
from ultralytics import YOLO

# Crear un modelo desde cero o cargar uno preentrenado
model = YOLO("yolov8m.pt")  # Puedes usar yolov8n.pt para un modelo más pequeño

# Entrenar el modelo
model.train(
    data="/content/drive/MyDrive/YOLOv8/dataset.yaml",  # Ruta al archivo dataset.yaml
    epochs=100,
    batch=8,
    imgsz=640,
    optimizer="AdamW",
    cos_lr=True
)


In [ ]:
# Validar el modelo entrenado
metrics = model.val()
print(metrics)


In [ ]:
# Realizar predicciones en imágenes de validación
results = model.predict(
    source="/content/drive/MyDrive/YOLOv8/datasets/val/images",
    save=True  # Guarda las predicciones en imágenes
)

# Mostrar los resultados
from IPython.display import Image, display
import glob

# Mostrar las primeras 5 imágenes con predicciones
predicted_images = glob.glob("runs/detect/predict/*.jpg")
for image_path in predicted_images[:5]:
    display(Image(filename=image_path))


In [ ]:
def validar_conexiones(predicciones, posiciones_correctas):
    """
    Validar si las conexiones son correctas basado en las predicciones.

    predicciones: lista de objetos detectados
    posiciones_correctas: diccionario con las posiciones esperadas para cada cable
    """
    for prediccion in predicciones:
        clase = prediccion['class']  # Índice de clase
        bbox = prediccion['bbox']  # [x_center, y_center, width, height]

        if clase not in posiciones_correctas:
            return "Incorrecto"
        if not bbox_esta_en_posicion(bbox, posiciones_correctas[clase]):
            return "Incorrecto"
    return "Correcto"

# Implementa lógica para definir las posiciones correctas y verificar


In [ ]:
# Guardar el modelo entrenado
model_path = "/content/drive/MyDrive/YOLOv8/cable_detector.pt"
model.export(format="torchscript", path=model_path)
print(f"Modelo guardado en: {model_path}")


In [ ]:
import pandas as pd

# Extraer resultados
predictions = results.pandas().xyxy[0]  # DataFrame con predicciones
predictions.to_csv("/content/drive/MyDrive/YOLOv8/predictions.csv", index=False)
print("Resultados guardados en predictions.csv")
